# Setup

In [72]:
# setup including building the graph
import rdflib
import os
import dominate
from dominate.tags import *

g = rdflib.Graph()

# directory paths assume existence of the p-lod dir two levels up
result = g.parse("../../p-lod/p-lod-vocabulary.ttl", format="turtle")
result = g.parse("../../p-lod/p-lod-entities.ttl", format="turtle")
result = g.parse("../../p-lod/p-lod-named-entities.ttl", format="turtle")

# Build individual files

In [73]:
# get unique subjects
subjects = g.query(
    """SELECT DISTINCT ?s
       WHERE {
          ?s ?p ?o .
       } ORDER BY ?s""")

# build individual index.html files
for row in subjects:
    path = str(row.s).replace('http://digitalhumanities.umass.edu/p-lod/','').split('/')
    
    directory = path[0]
    entity = path[1]
    
    if not os.path.exists("../" + directory):
        os.makedirs("../" + directory)
        
    if not os.path.exists("../" + directory + "/" + entity):
        os.makedirs("../" + directory + "/" + entity)
        
    p_os = g.query("SELECT ?p ?o WHERE { <http://digitalhumanities.umass.edu/p-lod/"+ directory + "/" + entity + "> ?p ?o } ORDER BY ?p")
    #print(description)
    
    doc = dominate.document(title="Pompeii LOD: " + entity)

    with doc.head:
        link(rel='stylesheet', href='style.css')
        # script(type='text/javascript', src='script.js')

    with doc:
        h1("Pompeii LOD " + directory + ": " + entity)
        for p_o in p_os:
            p(i(str(p_o.p)))
            with p(style="margin-left:1em"):
                if str(p_o.o)[0:4] == 'http':
                    a(str(p_o.o),href = str(p_o.o).replace('http://digitalhumanities.umass.edu/p-lod',''))
                else:
                    span(str(p_o.o))
            p()

    # actually write the file
    file = open("../" + directory + "/" + entity + "/index.html", 'w')
    file.write(doc.render())
    file.close()

# Build main index.html file

In [74]:
# build /index.html
index = g.query(
    """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
       PREFIX p-lod-v: <http://digitalhumanities.umass.edu/p-lod/vocabulary/>
       SELECT DISTINCT ?s ?label
       WHERE {
          ?s ?p ?o .
          ?s rdfs:label ?label .
          ?s a ?type .
          ?type rdfs:subClassOf* p-lod-v:named-entity .
          OPTIONAL { ?s a p-lod-v:named-entity . }
       } ORDER BY ?s""")

doc = dominate.document(title="Pompeii LOD: " + entity)

with doc.head:
    link(rel='stylesheet', href='style.css')
    # script(type='text/javascript', src='script.js')

with doc:
    h1("Pompeii LOD " + directory + ": " + entity)
    with ol():
        for i in index:
            with li():
                a(str(i.label),href = str(i.s).replace('http://digitalhumanities.umass.edu/p-lod',''))



    
    
# actually write the file
file = open("../index.html", 'w')
file.write(doc.render())
file.close()